In [ ]:
import itertools
import ipywidgets as widgets
import numpy as np
from IPython.display import display
from railyard.assembler import readStacks, get_hash

In [ ]:
base = '../stacks/base.yaml'

In [ ]:
base_options = [
    '../stacks/base.yaml',
    '../stacks/base_gpu.yaml'
]

In [ ]:
base_names = [
    'None',
    '10.2'
]

In [ ]:
groups = [
    ['../stacks/Python-datascience.yaml', '../stacks/Python-dataviz.yaml'],
    ['../stacks/R.yaml'],
    ['../stacks/julia.yaml'],
    ['../stacks/octave.yaml'],
    ['../stacks/java.yaml', '../stacks/scala.yaml'],
    ['../stacks/cpp.yaml'],
    ['../stacks/bash.yaml']
]
groups_names = [
    'Python Data Science',
    'R',
    'Julia',
    'Octave',
    'Java+Scala',
    'C++',
    'Bash'
]

# Build your own Jupyter container

All containers come with Python and Jupyter kernel preinstalled. Add more languages/libraries from the list and get the container ready to run

In [ ]:
repo = 'labshare/polyglot-notebook'

# Generate buttons for base options
base_choice_buttons = {}
for i,g in enumerate(base_names):
    base_choice_buttons.update({widgets.Button(description=g): i})


base_choice = 0
additional_choices = [False] * len(groups)

# Generate buttons
buttons = {}
for i,g in enumerate(groups_names):
    buttons.update({widgets.Button(description=g): i})

output = widgets.Output()
output2 = widgets.Output()

panel1 = widgets.HBox([widgets.Label('CUDA:      ')] + list(base_choice_buttons.keys()))
panel2 = widgets.HBox([widgets.Label('Add packages:      ')] + list(buttons.keys()))
panel3 = widgets.HBox([widgets.Label('Use this container: '), output])

display(panel1, panel2, panel3,output2)

# Define button callbacks
def on_base_choices_button_clicked(b):
    output.clear_output()
    output2.clear_output()
    
    # Set new base choice in variable
    global base_choice
    base_choice = base_choice_buttons[b]
    
    # Update buttons
    # Clear previous selection
    for bb in base_choice_buttons.keys():
        bb.icon = ''
        bb.button_style=''

    b.icon = 'check'
    b.button_style='warning'

    with output:
        h = get_hash(readStacks(base_options[base_choice],([item for sublist in np.array(groups)[additional_choices].tolist() for item in sublist])))
        print(repo + ':' + h)
    with output2:
        h = get_hash(readStacks(base_options[base_choice],([item for sublist in np.array(groups)[additional_choices].tolist() for item in sublist])))
        print('\n\n')
        print('Test locally:')
        print(f'docker run -it -p 8888:8888 {"--runtime=nvidia " if base_choice!=0 else ""}{repo}:{h}')

def on_additional_choices_button_clicked(b):
    global base_choice
    
    output.clear_output()
    output2.clear_output()
    index = buttons[b]
    additional_choices[index] = not additional_choices[index]
    if additional_choices[index]:
        b.icon = 'check'
        b.button_style='warning'
    else:
        b.icon = ''
        b.button_style=''
    with output:
        h = get_hash(readStacks(base_options[base_choice],([item for sublist in np.array(groups)[additional_choices].tolist() for item in sublist])))
        print(repo + ':' + h)
    with output2:
        h = get_hash(readStacks(base_options[base_choice],([item for sublist in np.array(groups)[additional_choices].tolist() for item in sublist])))
        print('\n\n')
        print('Test locally:')
        print(f'docker run -it -p 8888:8888 {"--runtime=nvidia " if base_choice!=0 else ""}{repo}:{h}')

for b in base_choice_buttons.keys():
    b.on_click(on_base_choices_button_clicked)

for b in buttons.keys():
    b.on_click(on_additional_choices_button_clicked)

with output:
        h = get_hash(readStacks(base,([item for sublist in np.array(groups)[additional_choices].tolist() for item in sublist])))
        print(repo + ':' + h)